<h1 align="center"  style="background-color:yellow;" style="font-family:verdana;"> ⬆️⬆️⬆️ If you find this note book helpful. <b>please upvote!</b> ⬆️⬆️⬆️ </h1>


<h1 align="center" style="background-color:powderblue;" style="font-size:300%;"> Google AI4Code – Understand Code in Python Notebooks   </h1>

<img src="https://tinyurl.com/4kjrs77t" align="center"> </img>


<h2 align="center" style="background-color:yellow;"  style="font-size:200%;"> import libraries </h2>


In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import json
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

In [2]:
class Config:
    train_dir = '../input/AI4Code/train'
    test_dir = '../input/AI4Code/test'
    order_csv = '../input/AI4Code/train_orders.csv'
    ancestor_csv = '../input/AI4Code/train_ancestors.csv'
    sub_csv = '../input/AI4Code/sample_submission.csv'

<h2 align="center" style="background-color:yellow;"  style="font-size:200%;"> load data </h2>


In [3]:
ancestor_df = pd.read_csv( Config.ancestor_csv)
sub_df = pd.read_csv( Config.sub_csv)
order_df = pd.read_csv( Config.order_csv)

In [4]:
ancestor_df.head()

,id,ancestor_id,parent_id
0,00001756c60be8,945aea18,NaN
1,00015c83e2717b,aa2da37e,317b65d12af9df
2,0001bdd4021779,a7711fde,NaN
3,0001daf4c2c76d,090152ca,NaN
4,0002115f48f982,272b483a,NaN


In [5]:
sub_df.head()

,id,cell_order
0,0009d135ece78d,ddfd239c c6cd22db 1372ae9b 90ed07ab 7f388a41 2...
1,0010483c12ba9b,54c7cab3 fe66203e 7844d5f8 5ce8863c 4a0777c4 4...
2,0010a919d60e4f,aafc3d23 80e077ec b190ebb4 ed415c3c 322850af c...
3,0028856e09c5b7,012c9d02 d22526d1 3ae7ece3 eb293dfc


In [6]:
order_df.head()

,id,cell_order
0,00001756c60be8,1862f0a6 448eb224 2a9e43d6 7e2f170a 038b763d 7...
1,00015c83e2717b,2e94bd7a 3e99dee9 b5e286ea da4f7550 c417225b 5...
2,0001bdd4021779,3fdc37be 073782ca 8ea7263c 80543cd8 38310c80 0...
3,0001daf4c2c76d,97266564 a898e555 86605076 76cc2642 ef279279 d...
4,0002115f48f982,9ec225f0 18281c6c e3b6b115 4a044c54 365fe576 a...


In [7]:
merged_df = order_df.merge( ancestor_df , on='id' )

In [8]:
merged_df.head()

,id,cell_order,ancestor_id,parent_id
0,00001756c60be8,1862f0a6 448eb224 2a9e43d6 7e2f170a 038b763d 7...,945aea18,NaN
1,00015c83e2717b,2e94bd7a 3e99dee9 b5e286ea da4f7550 c417225b 5...,aa2da37e,317b65d12af9df
2,0001bdd4021779,3fdc37be 073782ca 8ea7263c 80543cd8 38310c80 0...,a7711fde,NaN
3,0001daf4c2c76d,97266564 a898e555 86605076 76cc2642 ef279279 d...,090152ca,NaN
4,0002115f48f982,9ec225f0 18281c6c e3b6b115 4a044c54 365fe576 a...,272b483a,NaN


In [9]:
print( 'unique ids: ', merged_df.id.nunique())

unique ids:  139256


In [10]:
df_list =[]
def read_json( file ):
    file_name =  file.name.split('.')[0]
    try:
        df_ = pd.read_json( str( file )).reset_index().rename( {'index': 'cell_id'}, axis=1)
        df_['id'] = file_name
        df_list.append(df_)
    except:
        pass
    return df_list
files = Path( Config.train_dir ).glob('*.json')
results = Parallel(n_jobs=8)(delayed(read_json)(i) for i in tqdm(files))

0it [00:00, ?it/s]

In [11]:
df_list = []
for i in range(len(results)):
    df_ = results[i][0]
    df_list.append(df_)
df_raw = pd.concat( df_list, ignore_index=True)

In [12]:
df = merged_df.merge( df_raw , on='id')

In [13]:
df.head()

,id,cell_order,ancestor_id,parent_id,cell_id,cell_type,source
0,00015c83e2717b,2e94bd7a 3e99dee9 b5e286ea da4f7550 c417225b 5...,aa2da37e,317b65d12af9df,c417225b,code,import numpy as np # linear algebra\nimport pa...
1,00015c83e2717b,2e94bd7a 3e99dee9 b5e286ea da4f7550 c417225b 5...,aa2da37e,317b65d12af9df,51e3cd89,code,"input_dir = ""../input/"""
2,00015c83e2717b,2e94bd7a 3e99dee9 b5e286ea da4f7550 c417225b 5...,aa2da37e,317b65d12af9df,2600b4eb,code,"train_df = pd.read_csv(""../input/train.csv"")\n..."
3,00015c83e2717b,2e94bd7a 3e99dee9 b5e286ea da4f7550 c417225b 5...,aa2da37e,317b65d12af9df,75b65993,code,class_dict = defaultdict(int)\n\nkind_class_di...
4,00015c83e2717b,2e94bd7a 3e99dee9 b5e286ea da4f7550 c417225b 5...,aa2da37e,317b65d12af9df,cf195f8b,code,"print(""the number of images with no defects: {..."


In [14]:
%%time
df.to_csv( "AI4code_alldata_merged_raw.csv", index=False)

CPU times: user 4min 29s, sys: 8.68 s, total: 4min 38s
Wall time: 4min 38s


In [15]:
!zip AI4code_alldata_merged_raw.zip AI4code_alldata_merged_raw.csv

  adding: AI4code_alldata_merged_raw.csv (deflated 92%)


<div  class="alert alert-block alert-info">
    <h1 align='center'> <b>END of NOTEBOOK</b> </h1>
</div>